In [ ]:
# 📦 Install necessary libraries if not already installed (for Colab)
!pip install -q transformers ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.8 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
"""
Enhanced AI Text Generator
Improved version with better error handling, memory management, and features
"""

import os
import torch
import warnings
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForCausalLM
from IPython.display import Markdown, display
import gc

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

class AITextGenerator:
    def __init__(self, model_id='EleutherAI/gpt-neo-2.7B', seed=42):
        """
        Initialize the AI Text Generator

        Args:
            model_id (str): Hugging Face model identifier
            seed (int): Random seed for reproducibility
        """
        self.model_id = model_id
        self.generator = None
        self.tokenizer = None
        set_seed(seed)

    def setup_device(self):
        """Setup and return the appropriate device"""
        if torch.cuda.is_available():
            device = 0
            device_name = torch.cuda.get_device_name(0)
            memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"🚀 Using GPU: {device_name} ({memory_gb:.1f}GB)")
        else:
            device = -1
            print("🚀 Using CPU (GPU not available)")
        return device

    def load_model(self):
        """Load the model and tokenizer with error handling"""
        device = self.setup_device()

        try:
            print(f"📥 Loading model: {self.model_id}")

            # Load with optimized settings for memory efficiency
            self.generator = pipeline(
                "text-generation",
                model=self.model_id,
                device=device,
                torch_dtype=torch.float16 if device == 0 else torch.float32,
                trust_remote_code=True
            )

            # Load tokenizer separately for additional control
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            print("✅ Model loaded successfully!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            print("💡 Try using a smaller model like 'distilgpt2' or 'gpt2'")
            return False

    def generate_text(self, prompt, **kwargs):
        """
        Generate text with customizable parameters

        Args:
            prompt (str): Input text prompt
            **kwargs: Generation parameters
        """
        if not self.generator:
            print("❌ Model not loaded. Call load_model() first.")
            return None

        # Default generation parameters
        default_params = {
            'max_new_tokens': 256,  # Use max_new_tokens instead of max_length
            'do_sample': True,
            'temperature': 0.8,
            'top_k': 50,
            'top_p': 0.95,
            'repetition_penalty': 1.2,
            'pad_token_id': self.tokenizer.eos_token_id,
            'eos_token_id': self.tokenizer.eos_token_id,
            'early_stopping': True
        }

        # Update with user parameters
        params = {**default_params, **kwargs}

        try:
            print("🤖 Generating text...")

            # Generate text
            result = self.generator(prompt, **params)
            generated_text = result[0]["generated_text"]

            # Clean up GPU memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            return generated_text

        except Exception as e:
            print(f"❌ Error during generation: {e}")
            return None

    def interactive_generation(self):
        """Interactive text generation loop"""
        print("🎯 AI Text Generator Ready!")
        print("Commands: 'quit' to exit, 'params' to adjust settings")
        print("=" * 50)

        generation_params = {}

        while True:
            try:
                user_input = input("\n📝 Enter your prompt (or command): ").strip()

                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye!")
                    break

                elif user_input.lower() == 'params':
                    self.adjust_parameters(generation_params)
                    continue

                elif user_input.lower() == 'help':
                    self.show_help()
                    continue

                elif not user_input:
                    print("Please enter a valid prompt.")
                    continue

                # Generate text
                result = self.generate_text(user_input, **generation_params)

                if result:
                    print("\n" + "=" * 50)
                    if 'IPython' in str(type(get_ipython())):
                        display(Markdown(f"### 🧠 Generated Text:\n\n{result}"))
                    else:
                        print(f"🧠 Generated Text:\n\n{result}")
                    print("=" * 50)

            except KeyboardInterrupt:
                print("\n👋 Interrupted by user. Goodbye!")
                break
            except Exception as e:
                print(f"❌ Unexpected error: {e}")

    def adjust_parameters(self, params_dict):
        """Allow user to adjust generation parameters"""
        print("\n🎛️  Current parameters:")
        print(f"Temperature: {params_dict.get('temperature', 0.8)}")
        print(f"Max tokens: {params_dict.get('max_new_tokens', 256)}")
        print(f"Top-p: {params_dict.get('top_p', 0.95)}")

        try:
            temp = input("Enter temperature (0.1-2.0, current: 0.8): ").strip()
            if temp:
                params_dict['temperature'] = max(0.1, min(2.0, float(temp)))

            tokens = input("Enter max tokens (50-500, current: 256): ").strip()
            if tokens:
                params_dict['max_new_tokens'] = max(50, min(500, int(tokens)))

            top_p = input("Enter top-p (0.1-1.0, current: 0.95): ").strip()
            if top_p:
                params_dict['top_p'] = max(0.1, min(1.0, float(top_p)))

            print("✅ Parameters updated!")

        except ValueError:
            print("❌ Invalid input. Parameters unchanged.")

    def show_help(self):
        """Display help information"""
        help_text = """
        🆘 Help:
        - Enter any text prompt to generate AI content
        - 'params' - Adjust generation parameters
        - 'quit' - Exit the program
        - 'help' - Show this help message

        💡 Tips:
        - Be specific in your prompts
        - Try different temperatures (lower = more focused, higher = more creative)
        - Longer prompts often yield better results
        """
        print(help_text)

    def cleanup(self):
        """Clean up resources"""
        if hasattr(self, 'generator') and self.generator:
            del self.generator
        if hasattr(self, 'tokenizer') and self.tokenizer:
            del self.tokenizer

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        print("🧹 Resources cleaned up")

def main():
    """Main function to run the text generator"""
    # Available models (uncomment the one you want to use)
    models = {
        'small': 'distilgpt2',           # ~320MB, fast
        'medium': 'gpt2',                # ~500MB, balanced
        'large': 'gpt2-medium',          # ~1.4GB, better quality
        'xlarge': 'EleutherAI/gpt-neo-2.7B',  # ~10GB, high quality and better performance  
    }

    print("🤖 AI Text Generator")
    print("Available models:")
    for size, model in models.items():
        print(f"  {size}: {model}")

    # You can change this to any model from the dict above
    model_choice = 'xlarge'  # Change this as needed
    selected_model = models[model_choice]

    # Initialize generator
    generator = AITextGenerator(model_id=selected_model)

    try:
        # Load model
        if generator.load_model():
            # Start interactive generation
            generator.interactive_generation()
        else:
            print("Failed to load model. Exiting.")

    except KeyboardInterrupt:
        print("\n👋 Interrupted by user.")

    finally:
        # Cleanup
        generator.cleanup()

if __name__ == "__main__":
    main()
    

🤖 AI Text Generator
Available models:
  small: distilgpt2
  medium: gpt2
  large: gpt2-medium
  xlarge: EleutherAI/gpt-neo-2.7B
🚀 Using GPU: Tesla T4 (15.8GB)
📥 Loading model: EleutherAI/gpt-neo-2.7B


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Model loaded successfully!
🎯 AI Text Generator Ready!
Commands: 'quit' to exit, 'params' to adjust settings

📝 Enter your prompt (or command): What will be the future of Humans with AI?


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 Generating text...

🧠 Generated Text:

What will be the future of Humans with AI?

I have a strong feeling that this is going to be an interesting topic and not at all a simple answer. There are plenty of people who think that AI is a tool that will be used for good, and others who believe it’s a threat to human kind. So, I want to make sure that everyone understands where this will lead.

I’ve been in the AI industry for many years now and I’ve seen both sides. I started working on AI projects because I was fascinated by the technology, but I also understood that there are a lot of things that can go wrong. I’ve seen many attempts to make AIs that work on tasks that humans couldn’t do, or that were not interesting to humans. When I first started working on AI, I was a strong believer in the idea that artificial intelligence will improve the world and make it a better place. After all, if we could build robots that could do things like read and write, or draw, or play video games, th